In [ ]:
import pandas as pd
import numpy as np
import holidays
import json

In [ ]:
us_holidays = holidays.US()

In [ ]:
dv = '../data/denver_crime.csv'
weather = '../data/selected_weather_data.csv'
dv_json = '../data/denver_related.json'

In [ ]:
parse_dates = ['FIRST_OCCURRENCE_DATE', 'LAST_OCCURRENCE_DATE', 'REPORTED_DATE']
useful_cols =['OFFENSE_TYPE_ID','OFFENSE_CATEGORY_ID', 'FIRST_OCCURRENCE_DATE', 'LAST_OCCURRENCE_DATE', 'REPORTED_DATE', 
            'INCIDENT_ADDRESS','GEO_LON','GEO_LAT', 'NEIGHBORHOOD_ID', 'IS_TRAFFIC']
dvdf = pd.read_csv(dv, parse_dates=parse_dates, usecols=useful_cols)

dvdf.columns = map(str.lower, dvdf.columns)
dvdf.rename(columns={"offense_type_id": "crime_type", "offense_category_id": "crime_category",
                    'geo_lon': 'longitude', 'geo_lat': 'latitude',
                    'neighborhood_id': 'neighbourhood',
                    'incident_address': 'address'}, inplace=True)

In [ ]:
# Dealing with empty address
dvdf['address'] = dvdf['address'].apply(lambda x: x if x is not np.nan else 'NOT APPLICABLE')

In [ ]:
# Empty lat/lon
dvdf['latitude'] = dvdf['latitude'].apply(lambda x: x if not np.isnan(x) else 0)
dvdf['longitude'] = dvdf['longitude'].apply(lambda x: x if not np.isnan(x) else 0)

In [ ]:
# Weather
weather_df = pd.read_csv(weather, parse_dates = ['datetime'])
dvdf['tmp'] = pd.DatetimeIndex(dvdf['first_occurrence_date']).round('H')
dvdf['city']= 'Denver'
dvdf = dvdf.merge(weather_df, left_on=['city', 'tmp'], right_on =['city_name', 'datetime'], how='left')
dvdf.drop(columns=['tmp', 'datetime', 'city_name'], inplace=True)

In [ ]:
# Crime rate
# 2723000 is the average population from 2015 to 2020
dvdf['crime_rate'] = (len(dvdf.index) / 6) * (10000 / 2723000)

In [ ]:
dvdf['crime_date'] = pd.DatetimeIndex(dvdf['first_occurrence_date']).date
dvdf['day_of_week'] = pd.DatetimeIndex(dvdf['first_occurrence_date']).dayofweek
dvdf['week_of_year'] = pd.DatetimeIndex(dvdf['first_occurrence_date']).week
dvdf['quarter'] = pd.DatetimeIndex(dvdf['first_occurrence_date']).quarter

# holiday
weekend = pd.DataFrame({'day_of_week':[0,1,2,3,4,5,6],'weekend':[False,False,False,False,False,True,True]})
dvdf = dvdf.merge(weekend, how='left', left_on=['day_of_week'], right_on = ['day_of_week'])
dvdf['holiday'] = dvdf['crime_date'].apply(lambda x: us_holidays.get(x) is not None)
dvdf['holiday_name'] = dvdf['crime_date'].apply(lambda x: 'NOT APPLICABLE' if us_holidays.get(x) is None else us_holidays.get(x))

In [ ]:
# crime severity index
with open(dv_json) as f:
    dv_dict = json.load(f)

dvdf['crime_severity_c'] = dvdf['crime_category'].apply(lambda x: True if x in dv_dict['violent-crime'] else False)
dvdf['crime_severity_t'] = dvdf['crime_type'].apply(lambda x: True if x in dv_dict['violent-crime'] else False)
dvdf['crime_severity'] = dvdf.crime_severity_c | dvdf.crime_severity_t
dvdf.drop(columns=['crime_severity_t', 'crime_severity_c'], inplace=True)
dvdf['crime_severity'].replace([True, False], ['violent', 'non-violent'], inplace = True)

In [ ]:
# is nighttime
dvdf['is_nighttime'] = pd.DatetimeIndex(dvdf['first_occurrence_date']).hour
dvdf['is_nighttime'] = dvdf['is_nighttime'].apply(lambda x: True if x >= 21 or x<=5 else False)

In [ ]:
# is fatal
dvdf['is_fatal_c'] = dvdf['crime_category'].apply(lambda x: True if x in dv_dict['fatal-crime'] else False)
dvdf['is_fatal_t'] = dvdf['crime_type'].apply(lambda x: True if x in dv_dict['fatal-crime'] else False)
dvdf['is_fatal'] = dvdf.is_fatal_c | dvdf.is_fatal_t
dvdf.drop(columns=['is_fatal_c', 'is_fatal_t'], inplace=True)

In [ ]:
dvdf.rename(columns={'first_occurrence_date': 'first_occurrence_datetime', 
                     'last_occurrence_date': 'last_occurrence_datetime',
                    'reported_date': 'reported_datetime'}, inplace=True)

# Fix empty last occr
dvdf['last_occurrence_datetime'] = dvdf[['first_occurrence_datetime', 'last_occurrence_datetime']].apply(
    lambda x: x.first_occurrence_datetime if pd.isnull(x.last_occurrence_datetime) else x.last_occurrence_datetime,
axis=1)

In [ ]:
# Split datetime to date and time

dvdf['first_occurrence_date'] = pd.DatetimeIndex(dvdf['first_occurrence_datetime']).date
dvdf['first_occurrence_time'] = pd.DatetimeIndex(dvdf['first_occurrence_datetime']).time

dvdf['last_occurrence_date'] = pd.DatetimeIndex(dvdf['last_occurrence_datetime']).date
dvdf['last_occurrence_time'] = pd.DatetimeIndex(dvdf['last_occurrence_datetime']).time

dvdf['reported_date'] = pd.DatetimeIndex(dvdf['reported_datetime']).date
dvdf['reported_time'] = pd.DatetimeIndex(dvdf['reported_datetime']).time

dvdf.drop(columns=['first_occurrence_datetime', 'last_occurrence_datetime', 'reported_datetime'], inplace=True)

In [ ]:
dvdf.head()

In [ ]:
dvdf.columns

In [ ]:
dvdf.to_csv('../out/'+'dv.csv', index=False)